# Imports

In [ ]:
from aoc import *
import re
import os
import itertools
import math
from aocd.models import Puzzle as AOCDPuzzle


def Puzzle(day, year=2024):
    return AOCDPuzzle(year=year, day=day)

# Day 1 - Historian Hysteria

In [ ]:
p = Puzzle(1)
p

In [ ]:
lines = p.input_data.splitlines()
pairs = [tuple(int(i) for i in line.split("  ")) for line in lines]
lists = [sorted([tpl[i] for tpl in pairs]) for i in range(2)]
p.answer_a = sum(abs(a - b) for a, b in zip(*lists))

In [ ]:
from collections import Counter

counts = Counter(lists[1])
p.answer_b = sum(val * counts[val] for val in lists[0])

# Day 2 - Red-Nosed Reports

In [ ]:
p = Puzzle(year=2024, day=2)
p

In [ ]:
reports = [vector(line) for line in p.input_data.splitlines()]


def is_safe(report):
    diffs = [(b - a) for a, b in itertools.pairwise(report)]
    return all(1 <= abs(d) <= 3 for d in diffs) and (
        all(d > 0 for d in diffs) or all(d < 0 for d in diffs)
    )


def signum(i):
    return -1 if i < 0 else 1 if i > 0 else 0


def dampen(report):
    return is_safe(report) or (
        any(is_safe(report[:i] + report[i + 1 :]) for i in range(len(report)))
    )


p.answer_a = len([r for r in reports if is_safe(r)])
p.answer_b = len([r for r in reports if dampen(r)])

# Day 3 - Null It Over

In [ ]:
p = Puzzle(day=3)
p

In [ ]:
p.answer_a = sum(
    int(x) * int(y) for x, y in re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", p.input_data)
)

In [ ]:
def null_it_over_p2(input: str):
    instrux = re.findall(r"mul\((\d{1,3}),(\d{1,3})\)|(do|don't)\(\)", input)
    enabled = True
    sum = 0
    for x, y, mode in instrux:
        if mode == "do":
            enabled = True
        elif mode == "don't":
            enabled = False
        elif enabled:
            sum += int(x) * int(y)
    return sum


assert 48 == null_it_over_p2(
    """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5)))"""
)

p.answer_b = null_it_over_p2(p.input_data)

# Day 4 - Ceres Search

In [ ]:
p = Puzzle(4)
p

In [ ]:
SEARCH_DIR = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]


def word_search(grid, p, dir, word) -> bool:
    for i in range(len(word)):
        q = (p[0] + dir[0] * i, p[1] + dir[1] * i)
        if grid.get(q) != word[i]:
            return False
    return True


def find_words(input: str, word="XMAS") -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    xes = [p for p, c in grid.items() if c == "X"]
    answer = 0
    for x in xes:
        answer += sum(word_search(grid, x, dir, word) for dir in SEARCH_DIR)
    return answer


def is_x_mas(grid, a) -> bool:
    # Only diagonals!
    neighbors = [(-1, -1), (1, 1), (-1, 1), (1, -1)]
    letters = "".join(grid.get((a[0] + n[0], a[1] + n[1]), "") for n in neighbors)
    return len(letters) == 4 and letters in ("MSMS", "SMSM", "SMMS", "MSSM")


def find_x_mas(input: str) -> int:
    grid = dict(
        ((y, x), c)
        for y, line in enumerate(input.splitlines())
        for x, c in enumerate(line)
    )
    a_s = [p for p, c in grid.items() if c == "A"]
    return len([a for a in a_s if is_x_mas(grid, a)])


assert 18 == find_words(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_a = find_words(p.input_data)

assert 9 == find_x_mas(
    """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
)

p.answer_b = find_x_mas(p.input_data)